In [4]:
import pandas as pd 
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sbn
import pickle as pkl 

pd.set_option('display.max_columns', None)

In [10]:
# Metadata for cell lines (note there is one entry per compound/moa/target/structure combination, so some compounds appear more than once)
druginfo = pd.read_csv('../data/raw/compoundinfo_beta.txt', sep='\t')
druginfo2 = druginfo[lambda x: ~x.target.isna()]
druginfo2.head()

,pert_id,cmap_name,target,moa,canonical_smiles,inchi_key,compound_aliases
602,BRD-A50311610,meclizine,NR1I3,CAR agonist,Cc1cccc(CN2CCN(CC2)C(c2ccccc2)c2ccc(Cl)cc2)c1,OCJYIGYOJCODJL-UHFFFAOYSA-N,meclozine
603,BRD-K30743633,TCPOBOP,NR1I3,CAR agonist,Clc1cnc(Oc2ccc(Oc3ncc(Cl)cc3Cl)cc2)c(Cl)c1,BAFKRPOFIYPKBQ-UHFFFAOYSA-N,tcpobop
604,BRD-K39381259,DMH1,ACVR1,ALK inhibitor,CC(C)Oc1ccc(cc1)-c1cnc2c(cnn2c1)-c1ccnc2ccccc12,JMIFGARJSWXZSH-UHFFFAOYSA-N,DMH-1
605,BRD-K43002773,GDC-0068,AKT3,Akt inhibitor,C[C@@H]1C[C@H](C2=C1C(=NC=N2)N3CCN(CC3)C(=O)[C...,GRZXWCHAXNAUHY-NSISKUIASA-N,ipatasertib
606,BRD-K43002773,GDC-0068,AKT1,Akt inhibitor,C[C@@H]1C[C@H](C2=C1C(=NC=N2)N3CCN(CC3)C(=O)[C...,GRZXWCHAXNAUHY-NSISKUIASA-N,ipatasertib


In [12]:
with open('../../../DDS_graph.pkl', 'rb') as f: 
        DDS_graph = pkl.load(f)

with open('../../../PPI_graph.pkl', 'rb') as f: 
    PPI_graph = pkl.load(f)

drugs = DDS_graph['nodelist']
genes = PPI_graph['nodelist']
all2symb = PPI_graph['genemap']

In [35]:
len(drugs)

1224

In [36]:
len(genes)

13565

In [9]:
# how many drugs in our drug-drug network have targets in cmap meta? 
druginfo2 = druginfo[lambda x: ~x.target.isna()]
drug_overlap = set(druginfo2.cmap_name.str.lower().unique()).intersection(set(drugs))
len(drug_overlap)

755

In [16]:
# how many of the targets overlap with genes in our network? 
gene_overlap = set(druginfo2.target.str.upper().map(all2symb).values).intersection(set(genes))
len(gene_overlap)

848

In [20]:
# if we filter to targets that are in our gene graph... how many drugs in our graph have protein targets?
druginfo3 = druginfo2.assign(target2 = lambda x: x.target.str.upper().map(all2symb))
druginfo3 = druginfo3[lambda x: x.target2.isin(genes)]
print('# dropped obs:', druginfo2.shape[0] - druginfo3.shape[0])
drug_overlap2 = set(druginfo3.cmap_name.str.lower().unique()).intersection(set(drugs))
len(drug_overlap2)

# dropped obs: 166


745

In [37]:
# what are the moa (mode of action) types we have? 
druginfo3.fillna('yourmom').groupby('moa').count()['target'].sort_values().tail(10)

moa
KIT inhibitor                        114
FLT3 inhibitor                       121
Acetylcholine receptor antagonist    138
Cyclooxygenase inhibitor             149
PDGFR inhibitor                      164
Adrenergic receptor antagonist       194
VEGFR inhibitor                      199
Dopamine receptor antagonist         216
Dopamine receptor agonist            216
Serotonin receptor antagonist        276
Name: target, dtype: int64

In [27]:
druginfo4 = druginfo3.assign(moa_type = lambda x: x.moa.str.lower().str.split(' '))
druginfo4.moa_type = [x[-1] for x in druginfo4.moa_type]
druginfo4.head()

,pert_id,cmap_name,target,moa,canonical_smiles,inchi_key,compound_aliases,target2,moa_type
602,BRD-A50311610,meclizine,NR1I3,CAR agonist,Cc1cccc(CN2CCN(CC2)C(c2ccccc2)c2ccc(Cl)cc2)c1,OCJYIGYOJCODJL-UHFFFAOYSA-N,meclozine,NR1I3,agonist
603,BRD-K30743633,TCPOBOP,NR1I3,CAR agonist,Clc1cnc(Oc2ccc(Oc3ncc(Cl)cc3Cl)cc2)c(Cl)c1,BAFKRPOFIYPKBQ-UHFFFAOYSA-N,tcpobop,NR1I3,agonist
604,BRD-K39381259,DMH1,ACVR1,ALK inhibitor,CC(C)Oc1ccc(cc1)-c1cnc2c(cnn2c1)-c1ccnc2ccccc12,JMIFGARJSWXZSH-UHFFFAOYSA-N,DMH-1,ACVR1,inhibitor
605,BRD-K43002773,GDC-0068,AKT3,Akt inhibitor,C[C@@H]1C[C@H](C2=C1C(=NC=N2)N3CCN(CC3)C(=O)[C...,GRZXWCHAXNAUHY-NSISKUIASA-N,ipatasertib,AKT3,inhibitor
606,BRD-K43002773,GDC-0068,AKT1,Akt inhibitor,C[C@@H]1C[C@H](C2=C1C(=NC=N2)N3CCN(CC3)C(=O)[C...,GRZXWCHAXNAUHY-NSISKUIASA-N,ipatasertib,AKT1,inhibitor


In [28]:
druginfo4.groupby('moa_type').count()['target'].sort_values()

moa_type
intercalator            1
cells                   1
sympatholytic           1
antiviral               1
antifibrinolytic        1
compounds               1
potentiator             1
corrector               1
stabilizer              1
activity                1
steroid                 1
antiarrhythmic          2
anesthetic              2
anti-hcve2              3
progestin               3
scavenger               3
antifolate              3
antihistamine           3
drug                    3
acid                    3
inducer                 4
effects                 4
vasodilator             6
antioxidant             6
b                       7
sulfonylurea            8
hormone                 8
neurotransmitter        9
diuretic                9
precursor              10
analog                 11
immunosuppressant      13
antidepressant         14
sensitizer             14
enhancer               14
secretagogue           18
antiepileptic          26
ligand                 35
sti

In [31]:
# what drugs have the most targets? 
druginfo4.groupby('cmap_name').count()['target'].sort_values().tail(25)

cmap_name
LY-294002         28
zonisamide        28
asenapine         28
ponatinib         30
BMS-777607        30
cabergoline       30
pentobarbital     32
ellagic-acid      35
yohimbine         35
amitriptyline     36
clozapine         40
orantinib         42
ginkgolide-b      42
ursolic-acid      44
serotonin         48
dovitinib         50
tozasertib        56
dasatinib         60
guggulsterone     60
topiramate        66
bromocriptine     70
sunitinib         80
regorafenib       96
etomidate         96
sorafenib        132
Name: target, dtype: int64

In [34]:
# what targets are shared by the most drugs? 
druginfo4.groupby('target').count()['cmap_name'].sort_values().tail(25)

target
EGFR       54
MTOR       55
FGFR1      57
ADRB2      57
GABRA1     58
AR         61
FLT4       63
CHRM1      64
HTR2C      65
SLC6A2     69
PGR        73
SLC6A4     74
FLT1       75
ESR1       76
PDGFRB     77
KIT        83
FLT3       84
HTR1A      86
HRH1       88
PTGS1      95
PTGS2      99
KDR       105
HTR2A     112
NR3C1     112
DRD2      118
Name: cmap_name, dtype: int64